In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
#!pip install openpyxl xlrd
df_data = pd.read_excel('dataformdp.xlsm')

In [3]:
df_data.describe()

,Unnamed: 0,cell_z,ret,interSiteDist,rrcCongestionRate,rrcCongestionRateNeigh,taOvershootingFactor,lowRsrpSamplesRateEdge,numCellsHighOverlapHighRsrpSrcAgg,numCellsHighOverlapHighRsrpTgtAgg,...,nx_numCellsHighOverlapHighRsrpSrcAgg,nx_numCellsHighOverlapHighRsrpTgtAgg,nx_numTimesInterf,nx_trafficShare,nx_frequency,nx_ret_rel,nx_mechanical_tilt,nx_capacity,nx_coverage,nx_quality
count,1050.000000,1050.000000,1050.000000,1050.000000,1.050000e+03,1.050000e+03,1050.000000,1050.000000,1050.000000,1050.000000,...,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.0,1.050000e+03,1.050000e+03,1.050000e+03
mean,524.500000,0.328952,0.703571,0.102000,1.225053e-04,2.728095e-04,0.597076,0.073973,0.614412,0.543082,...,0.604481,0.530880,0.179301,0.855477,0.164000,0.508173,0.0,1.564583e-04,6.481312e-01,7.521674e-01
std,303.253195,0.027102,0.289075,0.025625,3.767788e-03,2.933974e-03,0.159884,0.117438,0.189916,0.275982,...,0.181949,0.260344,0.093424,0.290615,0.066845,0.233406,0.0,3.586272e-03,4.509015e-01,3.515366e-01
min,0.000000,0.280000,0.000000,0.060000,0.000000e+00,0.000000e+00,0.268029,0.000000,0.229167,0.119666,...,0.229167,0.119666,0.029562,0.186398,0.070000,0.000000,0.0,8.880293e-16,8.880293e-16,8.880293e-16
25%,262.250000,0.306667,0.500000,0.100000,0.000000e+00,2.263750e-35,0.500808,0.000000,0.476190,0.323859,...,0.473684,0.327033,0.113247,0.671992,0.110000,0.333333,0.0,8.880293e-16,8.880293e-16,5.128850e-01
50%,524.500000,0.326667,0.700000,0.100000,3.250000e-44,1.054164e-24,0.575602,0.000000,0.579855,0.478472,...,0.574468,0.474055,0.155464,0.845148,0.190000,0.500000,0.0,8.880293e-16,1.000000e+00,1.000000e+00
75%,786.750000,0.346667,0.900000,0.110000,1.758533e-33,1.174205e-15,0.660012,0.125000,0.702703,0.716617,...,0.693742,0.677306,0.219557,1.043394,0.190000,0.666667,0.0,8.880293e-16,1.000000e+00,1.000000e+00
max,1049.000000,0.380000,1.500000,0.140000,1.219961e-01,5.834595e-02,1.300705,0.800000,1.500000,1.521476,...,1.500000,1.487571,0.561323,2.171339,0.260000,1.000000,0.0,1.083916e-01,1.000000e+00,1.000000e+00


# Understanding the Data

Use Coverage, Quality and Capacity for state 

In [4]:
df_data.describe().coverage
non_float_rows = df_data[(~df_data['coverage'].astype(str).str.replace('.', '').str.replace('e-', '').str.isnumeric() &
                                 ~df_data['coverage'].astype(str).str.isdigit())]
print(len(non_float_rows))

0


/tmp/ipykernel_6994/2144402467.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  non_float_rows = df_data[(~df_data['coverage'].astype(str).str.replace('.', '').str.replace('e-', '').str.isnumeric() &


In [5]:
print(df_data[(~df_data['coverage'].astype(str).str.replace('.', '').str.replace('e-', '').str.isnumeric() &
                                 ~df_data['coverage'].astype(str).str.isdigit())].coverage)

Series([], Name: coverage, dtype: float64)


/tmp/ipykernel_6994/2618359493.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  print(df_data[(~df_data['coverage'].astype(str).str.replace('.', '').str.replace('e-', '').str.isnumeric() &


In [6]:
df_data.describe().capacity
non_float_rows = df_data[~df_data['capacity'].astype(str).str.replace('.', '').str.replace('e-', '').str.isnumeric()]
print(len(non_float_rows))

0


/tmp/ipykernel_6994/2318912692.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  non_float_rows = df_data[~df_data['capacity'].astype(str).str.replace('.', '').str.replace('e-', '').str.isnumeric()]


In [7]:
df_data.describe().quality
non_float_rows = df_data[~df_data['quality'].astype(str).str.replace('.', '').str.replace('e-', '').str.isnumeric()]
print(len(non_float_rows))

0


/tmp/ipykernel_6994/2577041903.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  non_float_rows = df_data[~df_data['quality'].astype(str).str.replace('.', '').str.replace('e-', '').str.isnumeric()]


# Columns to define a state

In [8]:
state_var = ['capacity','quality','coverage']

nx_state_var = [('nx_'+ i) for i in state_var]

print(nx_state_var)

['nx_capacity', 'nx_quality', 'nx_coverage']


# Discretizing the columns

In [9]:
df_json = pd.read_json('fileSelection.json')

# Display the DataFrame
print(df_json)

  column_name  no_of_bins  bin_size                          bin_list
0    capacity        10.0       NaN                               NaN
1     quality         NaN       0.5                               NaN
2    coverage         NaN       NaN  [[0, 0.2], [0.2, 0.5], [0.5, 1]]


# Discretisation

In [10]:
#Function to return Minimum and Maximum value of a state feature
def min_max_states(non_null_values_col, non_null_values_col_nx):
    return min(non_null_values_col.min(),non_null_values_col_nx.min()), max(non_null_values_col.max(),non_null_values_col_nx.max())

In [11]:
for i, row in df_json.iterrows():
    col_name = row['column_name']
    col_name_nx = 'nx_' + col_name
    non_null_values_col, non_null_values_col_nx = df_data[col_name].dropna(), df_data[col_name_nx].dropna()
    
    if row['no_of_bins'] is not None and not math.isnan(row['no_of_bins']):
        bins = int(row['no_of_bins'])
        min_value, max_value = min_max_states(non_null_values_col, non_null_values_col_nx)
        bin_size = (max_value - min_value) / bins
        bin = [min_value + i * bin_size for i in range(bins + 1)]
        df_data[col_name] = pd.cut(non_null_values_col, bins=bins, labels=False) + 1
        df_data[col_name_nx] = pd.cut(non_null_values_col_nx, bins=bins, labels=False) + 1
    
    elif row['bin_size'] is not None and not math.isnan(row['bin_size']):
        bin_size = row['bin_size']
        min_value, max_value = min_max_states(non_null_values_col, non_null_values_col_nx)
        bin = [min_value + i * bin_size for i in range(bins + 1)]
        df_data[col_name] = pd.cut(non_null_values_col, bins=bins, labels=False) + 1
        df_data[col_name_nx] = pd.cut(non_null_values_col_nx, bins=bins, labels=False) + 1
    
    elif len(row['bin_list'])>0:
        bin_list = row['bin_list']
        if isinstance(bin_list, list):
            bin_list = [interval for interval in bin_list if all(not math.isnan(value) for value in interval)]
        if bin_list:
            bin_intervals = [interval[0] for interval in bin_list] + [bin_list[-1][-1]]
            df_data[col_name] = pd.cut(non_null_values_col, bins=bins, labels=False) + 1
            df_data[col_name_nx] = pd.cut(non_null_values_col_nx, bins=bins, labels=False) + 1


In [15]:
#Modify the resultant state features in a single column as list of state features
df_data['current_state'] = df_data[state_var].apply(lambda row: list(row), axis=1)
df_data['next_state'] = df_data[nx_state_var].apply(lambda row: list(row), axis=1)

In [1]:
#Doing the same with comma delimiter
df_data['current_state'] = df_data.loc[:, state_var].apply(lambda row: ', '.join(map(str, row)), axis=1)
df_data['next_state'] = df_data.loc[:, nx_state_var].apply(lambda row: ', '.join(map(str, row)), axis=1)

NameError: name 'df_data' is not defined

In [16]:
#
file_path = 'discretisedFile.xlsm'
df_data.to_excel(file_path, index=False)